In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel

In [7]:
# deleting a column, reordering and then saving
import pandas as pd
import Utils.Reader as Reader

import scipy.sparse as sps
df = pd.read_csv(filepath_or_buffer='../data/rewatches.csv',
                         sep=",",
                         skiprows=1,
                         header=None,
                         dtype={0: int, 1: int,2: int},
                         engine='python')

Reader.df_col_normalize(df,2)
matrix_df=df.sort_values(by=[0,1])
matrix_df.to_csv('../output/out_2.csv', index=False)

In [1]:
# normalizing a column, reordering and then saving
import pandas as pd
import Utils.Reader as Reader

import scipy.sparse as sps
df = pd.read_csv(filepath_or_buffer='../data/rewatches.csv',
                 sep=",",
                 skiprows=1,
                 header=None,
                 dtype={0: int, 1: int,2: int},
                 engine='python')

df[2]= Reader.df_col_normalize(df,2)
df=df.sort_values(by=[0,1])
df.to_csv('../output/out_2.csv', index=False)

In [6]:
# normalizing a column with substitution, reordering and then saving
import pandas as pd
import Utils.Reader as Reader

import scipy.sparse as sps
matrix_df = pd.read_csv(filepath_or_buffer='../output/extended.csv',
                        sep=",",
                        skiprows=1,
                        header=None,
                        dtype={0: int, 1: int, 2: int},
                        engine='python')
matrix_df.columns = ["a","b","data"]
Reader.df_col_normalize(matrix_df,"data",{-1:-0.5,0:0.5})
matrix_df.head()

,a,b,data
0,0,0,-0.5
1,0,1,-0.5
2,0,2,-0.5
3,0,3,-0.5
4,0,4,-0.5


In [2]:
# 1 HOT ENCODER
import Utils.Reader as Reader
Reader.oneHotEncoder(name="Type", colsToDelete=["Data"],colstoOneHot=["Type"],dfPath='../data/data_ICM_length.csv')

In [2]:
# DF SPLITTER
import Utils.Reader as Reader
Reader.df_splitter("Rewatch",[0,10,20],dtype={0: int, 1: int},name="rewatches_total_",dfPath='../data/rewatches/rewatches_total.csv',colstoDefine=["UserID","Rewatch"])

In [1]:
# DF PREPROCESSING MODE
# 1-> Displayed (counts times a given Item has been displayed(in impressions list) to the user
# 2-> Extended (adds an interaction (-1) if a User has an item in impression list
# 3-> Count rewatches of each User-Item pair
# 4-> Count all rewatches of each user
import Utils.Reader as Reader
Reader.read_train_csr(matrix_path="../data/interactions_and_impressions.csv",preprocess=5,saving=True)

Unique UserID in the URM are 41629
Unique UserID in the URM and ICM are 41629
Preprocessing with mode: 5


Passing through all dataset to gather infos...:   0%|          | 4087/5826506 [00:02<59:29, 1630.97it/s]  


KeyboardInterrupt: 

# Evaluator data

In [3]:
from Utils.Evaluator import EvaluatorHoldout
import Utils.Reader as Reader

from Data_manager.split_functions.split_train_validation_random_holdout import\
    split_train_in_two_percentage_global_sample
URM_all=Reader.read_train_csr()
URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage=0.80)
evaluator_validation=EvaluatorHoldout(URM_test,cutoff_list=[10])



Unique UserID in the URM are 41629
Unique UserID in the URM and ICM are 41629
5826506
EvaluatorHoldout: Ignoring 1619 ( 3.9%) Users that have less than 1 test interactions


# Stacker

In [13]:
import Utils.Reader as Reader
stacked_URM, stacked_ICM = Reader.stacker(ICM_path='../data/extended.csv',ICM_values_to_change={})
stacked_URM,stacked_ICM

Number of tags	 3, Number of item-tag tuples 8741496
Unique UserID in the URM are 41629
Unique UserID in the URM and ICM are 41629
Unique ItemID in the URM are 24507
Unique ItemID in the URM and ICM are 24507
Unique FeatureID in the URM are 3


(<41632x24507 sparse matrix of type '<class 'numpy.float64'>'
 	with 1625298 stored elements in Compressed Sparse Row format>,
 <24507x41632 sparse matrix of type '<class 'numpy.float64'>'
 	with 1625298 stored elements in Compressed Sparse Row format>)

In [21]:
#1
# BEST MODEL UNTIL NOW
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
from Utils.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.Hybrid.ItemUserHybridKNNRecommender import ItemUserHybridKNNRecommender
import Utils.Reader as Reader


recommender_ItemKNNCFCBF = ItemUserHybridKNNRecommender(stacked_URM)
recommender_ItemKNNCFCBF.fit()

result_df, _ = evaluator_validation.evaluateRecommender(recommender_ItemKNNCFCBF)
result_df

ItemUserHybridKNNRecommender: URM Detected 2171 ( 4.9%) users with no interactions.
ItemKNNCFRecommender: URM Detected 2171 ( 4.9%) users with no interactions.
UserKNNCFRecommender: URM Detected 2171 ( 4.9%) users with no interactions.
Similarity column 24507 (100.0%), 1412.04 column/sec. Elapsed time 17.36 sec
Similarity column 44431 (100.0%), 1654.29 column/sec. Elapsed time 26.86 sec
Similarity column 24507 (100.0%), 1553.38 column/sec. Elapsed time 15.78 sec
[[-0.13466226 -0.25618118 -0.08125112 ... -0.31922135 -0.1737014
  -0.29992056]
 [-0.32696763 -0.32751796 -0.30443826 ... -0.34325665 -0.29201877
  -0.34325665]
 [ 0.4063995   0.35023814  0.686581   ... -0.16162403 -0.06355649
  -0.22183034]
 ...
 [ 0.2912843  -0.00217684 -0.20609328 ... -0.25894073 -0.33176446
  -0.18409103]
 [ 0.11316058  0.47458276  0.2551001  ... -0.23534372 -0.17210568
  -0.09334218]
 [ 0.1636323   0.30685624  0.38348177 ... -0.07325518 -0.27065837
  -0.22454101]]
[[-0.0422927  -0.0422927  -0.0422927  ... 

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.961109,0.0,0.961109,0.029587,8.936391,0.992118,0.057998,0.661118,4.416627,0.211056


In [22]:
#2
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
import Utils.Reader as Reader

recommender_ItemKNNCBF = ItemKNNCBFRecommender(URM_train, stacked_ICM)
recommender_ItemKNNCBF.fit()

result_df, _ = evaluator_validation.evaluateRecommender(recommender_ItemKNNCBF)
result_df

ItemKNNCBFRecommender: URM Detected 2 ( 0.0%) users with no interactions.
Similarity column 24507 (100.0%), 1780.24 column/sec. Elapsed time 13.77 sec


/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),
/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:270: RuntimeWarning: invalid value encountered in double_scalars
  ndcg_ = rank_dcg / ideal_dcg


EvaluatorHoldout: Processed 40010 (100.0%) in 42.98 sec. Users per second: 931


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.039963,0.064582,0.059108,0.021831,0.033745,0.126453,NaN,0.047685,0.241865,0.16155,...,0.961109,0.232458,0.961109,0.095146,11.228076,0.998961,0.215798,0.851475,1.262395,0.486741


In [23]:
#3
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
import Utils.Reader as Reader

recommender_ItemKNNCF = ItemKNNCFRecommender(stacked_URM)
recommender_ItemKNNCF.fit()

result_df, _ = evaluator_validation.evaluateRecommender(recommender_ItemKNNCF)
result_df

ItemKNNCFRecommender: URM Detected 2171 ( 4.9%) users with no interactions.
Similarity column 24507 (100.0%), 1640.76 column/sec. Elapsed time 14.94 sec
EvaluatorHoldout: Processed 40010 (100.0%) in 1.09 min. Users per second: 611


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.961109,0.0,0.961109,0.03146,9.028594,0.992525,0.070326,0.681917,3.576279,0.315728


# Submission Creator


In [ ]:
# SUBMITTING PREDICTION
import Utils.Writer as Writer
from Utils.Writer import NameRecommender

submit= Writer.Writer(NameRecommender=NameRecommender.ItemKNNCBF, URM=URM_all,stackedICM=stacked_ICM)
submit.makeSubmission()

In [20]:
# SENZA MAPPING
import numpy as np
import pandas as pd
import scipy.sparse as sps

URM_path="../data/interactions_and_impressions.csv"
ICM_path='../data/rewatches.csv'
URM_all_dataframe = pd.read_csv(filepath_or_buffer=URM_path,
                                sep=",",
                                skiprows=1,
                                header=None,
                                dtype={0: int, 1: int, 2: str, 3: int},
                                engine='python')

URM_all_dataframe.columns = ["UserID", "ItemID", "NULL", "Interaction"]
URM_all_dataframe["Interaction"] = URM_all_dataframe["Interaction"].replace({0: 1, 1: 0})
ICM_dataframe = pd.read_csv(filepath_or_buffer=ICM_path,
                            sep=",",
                            skiprows=1,
                            header=None,
                            dtype={0: int, 1: int, 2: int},
                            engine='python')
URM_all_dataframe.columns = ["UserID", "ItemID", "NULL", "Interaction"]
ICM_dataframe.columns = ["UserID", "ItemID", "FeatureID"]
intersection =np.intersect1d(URM_all_dataframe["UserID"],ICM_dataframe["UserID"])
ICM_dataframe = ICM_dataframe[ICM_dataframe["UserID"].isin(intersection)]
URM_all_dataframe = URM_all_dataframe[URM_all_dataframe["UserID"].isin(intersection)]

URM_all = sps.csr_matrix((URM_all_dataframe["Interaction"].values,
                          (URM_all_dataframe["UserID"].values, URM_all_dataframe["ItemID"].values)))  # always support a desired shape

ICM_all = sps.csr_matrix((np.ones(len(ICM_dataframe["ItemID"].values)),
                          (ICM_dataframe["ItemID"].values, ICM_dataframe["FeatureID"].values)))

ICM_all.data = np.ones_like(ICM_all.data)  # transfor array with all 1s if xisting val
stacked_URM = sps.vstack([URM_all, ICM_all.T])
stacked_URM = sps.csr_matrix(stacked_URM)

stacked_ICM = sps.csr_matrix(stacked_URM.T)